# Tarea 1a: generar texto

En este cuaderno, aprenderá a utilizar el modelo de lenguaje grande (LLM) para generar una respuesta por correo electrónico a un cliente que hizo comentarios negativos acerca de la calidad del servicio al cliente que recibió del ingeniero de soporte. En este cuaderno, generará un correo electrónico con una nota de agradecimiento a partir del correo electrónico anterior del cliente. Se utilizará el modelo Amazon Titan con la API de Amazon Bedrock con el cliente Boto3.

La petición que se utiliza en esta tarea se denomina petición sin entrenamiento previo. En esta, se describe la tarea o la salida deseada al modelo de lenguaje en un lenguaje sencillo. Luego, el modelo utiliza sus conocimientos y capacidades previamente entrenados para generar una respuesta o completar la tarea en función de solo la petición proporcionada.

#### Situación
Usted es Bob, un gerente de servicio al cliente en AnyCompany, y algunos de sus clientes no se sienten satisfechos con el servicio al cliente, por lo que realizan comentarios negativos en relación con el servicio proporcionado por los ingenieros de soporte. Ahora, desea responder a esos clientes y disculparse por el servicio deficiente para recuperar su confianza. Necesita contar con la ayuda de un LLM para generar correos electrónicos masivos que sean amigables con los clientes y estén personalizados en función de la sensación que experimentaron con los correos electrónicos anteriores.

## Tarea 1a.1: configuración del entorno

En esta tarea, establecerá el entorno.

In [1]:
#Create a service client by name using the default session.
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## Tarea 1a.2: generar texto

En esta tarea, preparará una entrada para que el servicio Amazon Bedrock genere un correo electrónico.

In [2]:
# create the prompt
prompt_data = """
Command: Write an email from Bob, Customer Service Manager, AnyCompany to the customer "John Doe" 
who provided negative feedback on the service provided by our customer support 
engineer"""

In [3]:
body = json.dumps({
    "inputText": prompt_data, 
    "textGenerationConfig":{
        "maxTokenCount":8192,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
        }
    }) 

A continuación, utilizará el modelo Amazon Titan.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Nota:** Amazon Titan admite una ventana de contexto de aproximadamente 4000 tokens y acepta los siguientes parámetros:
- `inputText`: petición al LLM
- `textGenerationConfig`: estos son los parámetros que considerará el modelo en el momento de generar las salidas.

La API de Amazon Bedrock ofrece una API `invoke_model` que acepta lo siguiente:
- `modelId`: este es el nombre de recurso de Amazon (Amazon Resource Name, ARN) del modelo de los diferentes modelos fundacionales disponibles en Amazon Bedrock
- `accept`: el tipo de solicitud de entrada
- `contentType`: el tipo de contenido de la salida
- `body`: una cadena json que incluye la petición y las configuraciones

Consulte la [documentación](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html) para conocer los ID de modelo de generación de texto disponibles.

## Tarea 1a.3: invocar el modelo de lenguaje Amazon Titan Large

En esta tarea, analizará de qué forma el modelo genera una salida en función de la petición creada anteriormente.

### Completar la generación de la salida

Este correo electrónico se genera con el modelo Amazon Titan, que comprende la solicitud de entrada y utiliza su conocimiento inherente de las diferentes modalidades. La solicitud a la API es simultánea y espera que el modelo genere la salida en su totalidad.

In [4]:
#invoke model
modelId = 'amazon.titan-text-express-v1' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"
try:

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    outputText = response_body.get('results')[0].get('outputText')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error


In [5]:
# The relevant portion of the response begins after the first newline character
# Below we print the response beginning after the first occurence of '\n'.

email = outputText[outputText.index('\n')+1:]
print(email)


Subject: Apology for Negative Feedback
Dear John Doe,

I am writing to express my deepest apologies for the negative feedback you provided regarding the service provided by our customer support engineer.

We take customer feedback seriously and always strive to deliver the highest quality service. It is disheartening to hear that we have fallen short of your expectations.

I would like to assure you that we are taking steps to address the issues you raised. Our customer support engineer has been thoroughly trained on how to handle customer interactions and provide exceptional service. We are also reviewing our processes to identify areas for improvement.

In addition, I would like to offer you a complimentary service to make up for the negative experience you had. Please let me know how you would like to proceed.

Once again, I apologize for any inconvenience you have experienced. We value your feedback and will use it to improve our services.

Sincerely,
Bob
Customer Service Manager
A

### Generación de la salida de streaming

Bedrock también admite que la salida se pueda transmitir, ya que el modelo la genera en forma de fragmentos. Para generar este correo electrónico se invoca el modelo con la opción de streaming. `invoke_model_with_response_stream` devuelve una propiedad `ResponseStream` que se puede leer.

In [6]:
# invoke model with response stream
output = []
try:
    
    response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
    stream = response.get('body')
    
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                text = chunk_obj['outputText']
                output.append(text)
                print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n')
                i+=1
            
except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

		**Chunk 1**

Subject: Apology for Negative Feedback
Dear John Doe,

I am writing to express my deepest apologies for the negative feedback you provided regarding the service provided by our customer support engineer.

We take customer feedback seriously

		**Chunk 2**
 and always strive to deliver the highest quality service. It is disheartening to hear that we have fallen short of your expectations.

I would like to assure you that we are taking steps to address the issues you raised. Our customer support engineer has been thoroughly trained on how to handle customer interactions and provide exceptional service. We are also reviewing our

		**Chunk 3**
 processes to identify areas for improvement.

In addition, I would like to offer you a complimentary service to make up for the negative experience you had. Please let me know how you would like to proceed.

Once again, I apologize for any inconvenience you have experienced. We value your feedback and will use it to improve our servi

El enfoque de transmisión con respuesta ayuda a obtener rápidamente la salida del modelo y permite que el servicio lo complete a medida que usted lee. Esto ayuda en los casos prácticos en los que se solicita al modelo que genere textos más largos. Luego, puede combinar todos los fragmentos generados para formar la salida completa y usarla en su caso práctico. 

In [ ]:
#combine output chunks
print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')
complete_output = ''.join(output)
print(complete_output)


Experimentó con el uso del SDK boto3, que proporciona una exposición básica a la API de Amazon Bedrock. Con esta API, vio el caso práctico sobre la generación de un correo electrónico para responder a los comentarios negativos de un cliente.

### Pruébelo usted mismo
- Cambie las peticiones para que se adapten al caso práctico específico y evalúe la salida de los diferentes modelos.
- Juegue con la longitud del token para comprender la latencia y la capacidad de respuesta del servicio.
- Aplique diferentes principios de ingeniería de peticiones para obtener mejores salidas.

### Limpieza

Ha completado este cuaderno. Para ir a la siguiente parte del laboratorio, complete estos pasos:

- Cierre este archivo de cuaderno y continúe con **Task1b.ipynb**.